In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [ ]:
# Define a data de início do novo projeto (entrada do sistema)
inicio_novo_projeto = pd.Timestamp("")
portfolio_novo_projeto = ()

In [ ]:
tabela_pcp = pd.read('')

In [ ]:
# Converte datas para datetime
date_cols = [f'Fim previsto do Projeto {i}' for i in range(1, 5)] + \
            [f'Fim estimado do Projeto {i}' for i in range(1, 5)] + \
            [f'Fim do Projeto Interno {i}' for i in range(1, 5)]
for col in date_cols:
    tabela_pcp[col] = pd.to_datetime(tabela_pcp[col], errors='coerce')

In [ ]:
# Lista de cargos a serem removidos
cargos_excluidos = ['Líder de Outbound', 'Coordenador de Negócios', 'Coordenador de Inovação Comercial', 'Gerente Comercial',
                    'Coordenador de Projetos', 'Coordenador de Inovação de Projetos', 'Gerente de Projetos']

# Filtrando a tabela para remover os cargos indesejados
tabela_pcp = tabela_pcp[~tabela_pcp['Cargo no núcleo'].isin(cargos_excluidos)]


In [ ]:
# Calcula disponibilidade estimada do analista
# Comecamos com 30h, diminuimos a partir de fatores comoo alocacoes de aprendizagem, assessorias, projetos
def calcular_disponibilidade(analista, data_inicio_novo_projeto):
    horas_disponiveis = 30

    # Aprendizagem = 5h, assessoria = 10h
    horas_disponiveis -= analista.get('N° Aprendizagens', 0) * 5
    horas_disponiveis -= analista.get('N° Assessoria', 0) * 10

    # Ajusta disponibilidade conforme proximidade da data de fim de um projeto (projetos 1-4)
    # Se falta mais de 2 semanas, 10h de alocacao
    # So consideramos 6 horas de alocacao caso falte entre 1-2 semanas
    # Para menos de 1 semana, consideramos nao alocacao
    for i in range(1, 5):
        fim_projeto = analista.get(f'Fim estimado do Projeto {i}', analista.get(f'Fim previsto do Projeto {i}', None))

        if pd.notnull(fim_projeto):
            dias_restantes = (fim_projeto - data_inicio_novo_projeto).days
            if dias_restantes > 14:
                horas_disponiveis -= 10
            if 7 < dias_restantes <= 14:
                horas_disponiveis -= 6

    # Projeto interno = 5h
    for i in range(1, 5):
        fim_projeto_interno = analista.get(f'Fim do Projeto Interno {i}')
        if pd.notnull(fim_projeto_interno):
            if fim_projeto_interno > data_inicio_novo_projeto:
                horas_disponiveis -= 5
        elif pd.notnull(analista.get(f'Início do Projeto Interno {i}')):
            horas_disponiveis -= 5
             

    # Ajusta conforme cargo no núcleo
    cargo = str(analista.get('Cargo no núcleo', '')).strip().upper()
    if cargo in ['SDR', 'HUNTER']:
        horas_disponiveis -= 10
    elif cargo == 'ANALISTA SÊNIOR':
        horas_disponiveis -= 5

    return horas_disponiveis

In [ ]:
# Função para calcular a Afinidade (nota de 0 a 10)
#Usamos a satisfacao esperada do portfolio, alem de capacidade e saude mental do analista
def calcular_afinidade(analista, portfolio_novo_projeto):
    # Satisfação esperada = Satisfação Média com o Portfólio
    satisfacao_total = 0
    projetos_encontrados = 0

    for i in range(1, 5):
        portfolio_projeto = analista.get(f'Portfólio do Projeto {i}', '').strip().upper()
        if portfolio_projeto == portfolio_novo_projeto.strip().upper():
            satisfacao_total += analista.get(f'Satisfação com o Projeto {i}', 8)  # Assume 8 se não informado
            projetos_encontrados += 1

    if projetos_encontrados > 0:
        satisfacao_portfolio = satisfacao_total / projetos_encontrados
    else:
        satisfacao_portfolio = 8  # Assume 8 se nenhum projeto correspondente for encontrado

    # Capacidade esperada = Validação média do Projeto
    validacao_projetos = [analista.get(f'Validação do Projeto {i}', np.nan) for i in range(1, 5)]
    capacidade = (pd.Series(validacao_projetos).mean(skipna=True) if not pd.isna(validacao_projetos).all() else 0) * 2

    # Saúde mental = Média entre percepção da carga e saúde mental na PJ
    sentimento_carga = analista.get('Como se sente em relação à carga', '').strip().upper()
    sentimento_map = {'SUBALOCADO': 10, 'ESTOU SATISFEITO': 5, 'SUPERALOCADO': 1}
    sentimento_nota = sentimento_map.get(sentimento_carga, 5)  # Se não estiver mapeado, assume 5
    saude_mental = analista.get('Saúde mental na PJ', 5)

    saude_final = (sentimento_nota + saude_mental) / 2

    # Nota final de afinidade é a média dos três critérios
    afinidade = (satisfacao_portfolio + capacidade + saude_final) / 3
    return afinidade

In [ ]:
# Calcula disponibilidade
tabela_pcp['Disponibilidade'] = tabela_pcp.apply(lambda row: calcular_disponibilidade(row, inicio_novo_projeto), axis=1)

In [ ]:
# Calcula afinidade
tabela_pcp['Afinidade'] = tabela_pcp.apply(calcular_afinidade, axis=1)

In [ ]:
# Ordena os membros pela Disponibilidade e Afinidade (ambas igualmente importantes)
tabela_pcp = tabela_pcp.sort_values(by=['Disponibilidade', 'Afinidade'], ascending=[False, False])

In [ ]:
# Exibe as colunas principais
print("Membros sugeridos (ordenados pela pontuação):")
print(tabela_pcp[['Membro', 'Disponibilidade', 'Afinidade']])